In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 8.6 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [2]:
df = pd.read_csv('sim_data_trunc.csv')

In [3]:
print(df.head())

         Date  Step  price_A      tv_A  web_A  price_B  tv_B  web_B  \
0  2022-01-02    52      5.0       0.0    0.0      5.0   0.0    0.0   
1  2022-01-09    53      5.0       0.0    0.0      5.0   0.0    0.0   
2  2022-01-16    54      5.0       0.0    0.0      5.0   0.0    0.0   
3  2022-01-23    55      5.0       0.0    0.0      5.0   0.0    0.0   
4  2022-01-30    56      5.0  125000.0    0.0      5.0   0.0    0.0   

   Total_Purchases_A  Total_Purchases_B  Total_Sales_A  Total_Sales_B  
0              618.0              243.0      3090000.0      1215000.0  
1              531.0              187.0      2655000.0       935000.0  
2              653.0              240.0      3265000.0      1200000.0  
3              521.0              207.0      2605000.0      1035000.0  
4              620.0              231.0      3100000.0      1155000.0  


In [4]:
# ... existing imports ...

# After df = pd.read_csv('sim_data_trunc.csv') and data checks

# Define column mappings for Meridian
coord_to_columns = load.CoordToColumns(
    time='Date',
    controls=['price_A', 'price_B'],
    kpi='Total_Sales_A',  # Target variable
    media=[
        'tv_A',
        'web_A',
        'tv_B', 
        'web_B'
    ],
    media_spend=[
        'tv_A',
        'web_A',
        'tv_B',
        'web_B'
    ],
    # Map other variables if available:
    # population='population_column', 
    # organic_media=['organic_channel'],
    # non_media_treatments=['Promo']
)

# Map media columns to channel names
media_to_channel = {
    'tv_A': 'TV Campaign A',
    'web_A': 'Digital Campaign A',
    'tv_B': 'TV Campaign B',
    'web_B': 'Digital Campaign B'
}

# Initialize data loader
loader = load.CsvDataLoader(
    csv_path='sim_data_trunc.csv',  # Your data file
    kpi_type='revenue',  # Using sales as KPI
    coord_to_columns=coord_to_columns,
    media_to_channel=media_to_channel,
    media_spend_to_channel=media_to_channel,  # Same mapping for spend
)

# Load and prepare data
data = loader.load()

# Configure model with reasonable priors
roi_mu = 0.3  # Prior belief for average ROI
roi_sigma = 0.2  # Uncertainty around ROI estimate
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma)
)
model_spec = spec.ModelSpec(prior=prior)

# Initialize and run model
mmm = model.Meridian(input_data=data, model_spec=model_spec)
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:1020: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/Users/michael/.venvs/pymmm_0_6_0/lib/python3.12/site-packages/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It wil

In [7]:
# After model fitting

# 1. Model diagnostics (correct API from docs)
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()  # Shows R-hat distribution
model_diagnostics.plot_trace()  # Shows chain convergence

# 2. Results summary (official method)
summary = summarizer.Summarizer(mmm).create_summary()
print(summarizer.Formatter().format_summary(summary))

# 3. Media contributions (verified approach)
visualizer.ChannelAttribution(mmm).plot_contribution_breakdown()

# 4. Budget optimization (as shown in docs)
optimizer = optimizer.BudgetOptimizer(mmm)
optimization_result = optimizer.optimize()
print(optimization_result.optimal_allocation)

MCMCSamplingError: MCMC sampling failed with a maximum R-hat value of 2582931046400.0.